In [2]:
#import bibliotheques python
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import joblib

In [3]:
df = joblib.load('cleaned_data.sav')

In [4]:
df

index       Id                Tags  \
0          0        4                [c#]   
1          1        9          [c#, .net]   
2          2       11                [c#]   
3          3       16                [c#]   
4          4       17   [mysql, database]   
...      ...      ...                 ...   
41163  41163  8057565             [mysql]   
41164  41164  8054165              [html]   
41165  41165  8064691  [javascript, html]   
41166  41166  8057607              [java]   
41167  41167  8067590        [javascript]   

                                                    Body  java   c#  \
0      convert decim doubl track bar form opac lang p...   0.0  1.0   
1      calcul someon age base datetim birthday given ...   0.0  1.0   
2      calcul relat given specif datetim display rela...   0.0  1.0   
3      fill dataset datat linq queri result expo linq...   0.0  1.0   
4      binari mysql store binari http en wikipedia wi...   0.0  0.0   
...                                                  ...   ...  ...   
41163  mysql updat inner join queri idea problem mysq...   0.0  0.0   
41164                        put form put form send form   0.0  0.0   
41165  pas along variabl xmlhttprequest send variabl ...   0.0  0.0   
41166  mock privat powermock still mock privat jndi u...   1.0  0.0   
41167  associ versus hash search googl folk recommend...   0.0  0.0   

       javascript  python  jquery  html  ...  regex  iphone  bash  \
0             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
1             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
2             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
3             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
4             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
...           ...     ...     ...   ...  ...    ...     ...   ...   
41163         0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
41164         0.0     0.0     0.0   1.0  ...    0.0     0.0   0.0   
41165         1.0     0.0     0.0   1.0  ...    0.0     0.0   0.0   
41166         0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
41167         1.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   

       objective-c  ios  windows  ruby  eclipse  tsql  database  
0              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
1              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
2              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
3              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
4              0.0  0.0      0.0   0.0      0.0   0.0       1.0  
...            ...  ...      ...   ...      ...   ...       ...  
41163          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41164          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41165          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41166          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41167          0.0  0.0      0.0   0.0      0.0   0.0       0.0  

[41168 rows x 34 columns]

# Training data

In [5]:
X = df.Body
Y = df.drop(columns = ['Id','index','Body','Tags'])

# Unsupervided Method

## LDA

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df = 0.95, min_df = 0.01 , stop_words = 'english')
X_fited = tfidf_vec.fit_transform(X)

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 10,max_iter=5,learning_method='online',learning_offset=50,random_state = 0).fit(X_fited)

In [7]:
 def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 1
display_topics(lda, tfidf_vec.get_feature_names(), no_top_words)

Topic 0:
string
Topic 1:
http
Topic 2:
date
Topic 3:
button
Topic 4:
command
Topic 5:
git
Topic 6:
list
Topic 7:
android
Topic 8:
sql
Topic 9:
java


## NMF

In [8]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=10, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
nmf.fit(X_fited)

NMF(alpha=0.1, init='nndsvd', l1_ratio=0.5, n_components=10, random_state=1)

In [9]:
no_top_words = 1
display_topics(nmf, tfidf_vec.get_feature_names(), no_top_words)

Topic 0:
http
Topic 1:
string
Topic 2:
android
Topic 3:
java
Topic 4:
list
Topic 5:
sql
Topic 6:
git
Topic 7:
date
Topic 8:
button
Topic 9:
cs


# Supervised Method

## Train/Test Split

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

## TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df = 0.95, min_df = 0.01 , stop_words = 'english')
X_train = tfidf_vec.fit_transform(X_train)
X_test = tfidf_vec.transform(X_test)

## Scores

In [14]:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# liste des 10 catégories
categories = list(Y.columns.values)
# liste des métrics utilisé
scores = ['f1','accuracy','precision','recall']
# liste des modèles utilisé
modeles = ['Logistic_Regression','SVC','Naive_Bayes','MLP']
# Tableaux qui vont acceuillir les score pour de chaque model pour l'approche One_vs_rest
Logistic_Regression = pd.DataFrame(columns = scores,index = categories)
SVC = pd.DataFrame(columns = scores,index = categories)
Naive_Bayes = pd.DataFrame(columns = scores,index = categories)
MLP = pd.DataFrame(columns = scores,index = categories)
LinearSVC = pd.DataFrame(columns = scores,index = categories)
#Tableau qui va acceuillir les scores pour l'approche Binary Relevance
Binary_relevance_score = pd.DataFrame(index=scores,columns=modeles)
#Tableau qui va acceuillir les scores pour l'approche LabelPowerset
Label_powerset_score = pd.DataFrame(index=scores,columns=modeles)
#Tableau qui va acceuillir les scores pour le model MLKNN
MLKNN_Score = pd.DataFrame(index=['MLKNN'],columns=scores)


## MLKNN

In [12]:
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=10)
# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(X_train).toarray()
Y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(X_test).toarray()
#parameter
parameters = {'k': range(1,6)}
# train
clf = GridSearchCV(MLkNN(), parameters, scoring='f1_macro')
clf.fit(X_train, Y_train)
#best param
print (clf.best_params_, clf.best_score_)
# predict
predictions_new = clf.predict(x_test)

/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarni

{'k': 4} 0.3458601189498147


In [13]:

MLKNN_accuracy = accuracy_score(y_test, predictions_new)
MLKNN_precision = precision_score(y_test, predictions_new,average='weighted')
MLKNN_recall = recall_score(y_test, predictions_new,average='weighted')

MLKNN_Score['accuracy']=MLKNN_accuracy
MLKNN_Score['precision']=MLKNN_precision
MLKNN_Score['recall']=MLKNN_recall

## Logistique Regression

### One vs Rest

In [14]:
f1=[]
accuracy = []
precision = []
recall = []


from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=6)),
            ])
for category in categories:
    print('**Processing {} Tag...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(X_train, y_train[category])
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(X_test)
    accuracy.append(accuracy_score(y_test[category], prediction))
    f1.append(metrics.f1_score(y_test[category], prediction))
    precision.append(precision_score(y_test[category], prediction))
    recall.append(recall_score(y_test[category], prediction))

**Processing java Tag...**
**Processing c# Tag...**
**Processing javascript Tag...**
**Processing python Tag...**
**Processing jquery Tag...**
**Processing html Tag...**
**Processing php Tag...**
**Processing android Tag...**
**Processing .net Tag...**
**Processing c++ Tag...**
**Processing sql Tag...**
**Processing css Tag...**
**Processing mysql Tag...**
**Processing string Tag...**
**Processing sql-server Tag...**
**Processing c Tag...**
**Processing git Tag...**
**Processing arrays Tag...**
**Processing asp.net Tag...**
**Processing linux Tag...**
**Processing regex Tag...**
**Processing iphone Tag...**
**Processing bash Tag...**
**Processing objective-c Tag...**
**Processing ios Tag...**
**Processing windows Tag...**
**Processing ruby Tag...**
**Processing eclipse Tag...**
**Processing tsql Tag...**
**Processing database Tag...**


In [15]:
Logistic_Regression['f1'] = f1
Logistic_Regression['accuracy'] = accuracy
Logistic_Regression['precision'] = precision
Logistic_Regression['recall'] = recall

### Binary Relevance

In [16]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
# initialize binary relevance multi-label classifier
classifier = BinaryRelevance(LogisticRegression(penalty = 'l2' ,max_iter = 200,solver='sag'))
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [17]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Binary_relevance_score['Logistic_Regression']=Score


### Label Powerset

In [18]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression(penalty = 'l2' ,max_iter = 200,solver='sag'))
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [20]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Label_powerset_score['Logistic_Regression']=Score


## Support vector Classifier

### Label Powerset

In [21]:
# using Label Powerset
from sklearn.svm import SVC
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(SVC())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [21]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Label_powerset_score['SVC']=Score


### Binary Relevance

In [22]:
# using binary relevance 
from skmultilearn.problem_transform import BinaryRelevance
# initialize binary relevance multi-label classifier
classifier = BinaryRelevance(SVC())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [23]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Binary_relevance_score['SVC']=Score


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### One vs Rest

In [24]:
f1=[]
accuracy = []
precision = []
recall = []

# Using pipeline for applying logistic regression and one vs rest classifier
SVC_pipeline = Pipeline([('clf', OneVsRestClassifier(SVC(), n_jobs= 1))])
SVC_one_vs_rest = []
for category in categories:
    print('**Processing {} Tag...**'.format(category))
    
    # Training logistic regression model on train data
    SVC_pipeline.fit(X_train, y_train[category])
    # calculating test accuracy
    prediction = SVC_pipeline.predict(X_test)
    accuracy.append(accuracy_score(y_test[category], prediction))
    f1.append(f1_score(y_test[category], prediction))
    precision.append(precision_score(y_test[category], prediction))
    recall.append(recall_score(y_test[category], prediction))

**Processing java Tag...**
**Processing c# Tag...**
**Processing javascript Tag...**
**Processing python Tag...**
**Processing jquery Tag...**
**Processing html Tag...**
**Processing php Tag...**
**Processing android Tag...**
**Processing .net Tag...**


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Processing c++ Tag...**
**Processing sql Tag...**
**Processing css Tag...**
**Processing mysql Tag...**
**Processing string Tag...**
**Processing sql-server Tag...**
**Processing c Tag...**
**Processing git Tag...**
**Processing arrays Tag...**
**Processing asp.net Tag...**
**Processing linux Tag...**
**Processing regex Tag...**
**Processing iphone Tag...**
**Processing bash Tag...**
**Processing objective-c Tag...**
**Processing ios Tag...**
**Processing windows Tag...**
**Processing ruby Tag...**
**Processing eclipse Tag...**
**Processing tsql Tag...**


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Processing database Tag...**


In [25]:
SVC['f1'] = f1
SVC['accuracy'] = accuracy
SVC['precision'] = precision
SVC['recall'] = recall

## MultinomialNB

### One vs Rest

In [26]:
f1=[]
accuracy = []
precision = []
recall = []

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
# Using pipeline for applying logistic regression and one vs rest classifier
MultinomialNB_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
NB_One_vs_Rest = []
for category in categories:
    print('**Processing {} Tag...**'.format(category))
    
    # Training logistic regression model on train data
    MultinomialNB_pipeline.fit(X_train, y_train[category])
    # calculating test accuracy
    prediction = MultinomialNB_pipeline.predict(X_test)
    accuracy.append(accuracy_score(y_test[category], prediction))
    f1.append(f1_score(y_test[category], prediction))
    precision.append(precision_score(y_test[category], prediction))
    recall.append(recall_score(y_test[category], prediction)) 

**Processing java Tag...**
**Processing c# Tag...**
**Processing javascript Tag...**
**Processing python Tag...**
**Processing jquery Tag...**
**Processing html Tag...**
**Processing php Tag...**
**Processing android Tag...**
**Processing .net Tag...**
**Processing c++ Tag...**
**Processing sql Tag...**
**Processing css Tag...**
**Processing mysql Tag...**
**Processing string Tag...**
**Processing sql-server Tag...**
**Processing c Tag...**
**Processing git Tag...**
**Processing arrays Tag...**
**Processing asp.net Tag...**


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Processing linux Tag...**
**Processing regex Tag...**
**Processing iphone Tag...**
**Processing bash Tag...**
**Processing objective-c Tag...**
**Processing ios Tag...**


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Processing windows Tag...**
**Processing ruby Tag...**


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Processing eclipse Tag...**
**Processing tsql Tag...**
**Processing database Tag...**


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
Naive_Bayes['f1'] = f1
Naive_Bayes['accuracy'] = accuracy
Naive_Bayes['precision'] = precision
Naive_Bayes['recall'] = recall

### Binary Relevance

In [28]:
# using binary relevance 
from skmultilearn.problem_transform import BinaryRelevance
# initialize binary relevance multi-label classifier
classifier = BinaryRelevance(MultinomialNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [29]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Binary_relevance_score['Naive_Bayes']=Score


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Label Powerset

In [30]:
# using Label Powerset
from sklearn.svm import SVC
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(MultinomialNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [31]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Label_powerset_score['Naive_Bayes']=Score


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## MLP Classifier

### One vs Rest

In [32]:
f1=[]
accuracy = []
precision = []
recall = []

from sklearn.neural_network import MLPClassifier
# Using pipeline for applying logistic regression and one vs rest classifier
MLPClassifier_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(MLPClassifier(), n_jobs=-1)),
            ])
MLP_One_vs_Rest = []
for category in categories:
    print('**Processing {} Tag...**'.format(category))
    
    # Training logistic regression model on train data
    MLPClassifier_pipeline.fit(X_train, y_train[category])
    # calculating test accuracy
    prediction = MLPClassifier_pipeline.predict(X_test)
    accuracy.append(accuracy_score(y_test[category], prediction))
    f1.append(f1_score(y_test[category], prediction))
    precision.append(precision_score(y_test[category], prediction))
    recall.append(recall_score(y_test[category], prediction))

**Processing java Tag...**
**Processing c# Tag...**
**Processing javascript Tag...**
**Processing python Tag...**
**Processing jquery Tag...**
**Processing html Tag...**
**Processing php Tag...**
**Processing android Tag...**
**Processing .net Tag...**
**Processing c++ Tag...**
**Processing sql Tag...**
**Processing css Tag...**
**Processing mysql Tag...**
**Processing string Tag...**
**Processing sql-server Tag...**
**Processing c Tag...**
**Processing git Tag...**
**Processing arrays Tag...**
**Processing asp.net Tag...**
**Processing linux Tag...**
**Processing regex Tag...**
**Processing iphone Tag...**
**Processing bash Tag...**
**Processing objective-c Tag...**
**Processing ios Tag...**
**Processing windows Tag...**
**Processing ruby Tag...**
**Processing eclipse Tag...**
**Processing tsql Tag...**
**Processing database Tag...**


In [33]:
MLP['f1'] = f1
MLP['accuracy'] = accuracy
MLP['precision'] = precision
MLP['recall'] = recall

### Binary Relevance

In [34]:
# using binary relevance 
from skmultilearn.problem_transform import BinaryRelevance
# initialize binary relevance multi-label classifier
classifier = BinaryRelevance(MultinomialNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [35]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Binary_relevance_score['MLP']=Score

/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Label Powerset

In [36]:
# using Label Powerset
from sklearn.svm import SVC
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(MultinomialNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [37]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Label_powerset_score['MLP']=Score

/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Linear SVC

In [16]:
f1=[]
accuracy = []
precision = []
recall = []
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
# Using pipeline for applying logistic regression and one vs rest classifier
LinearSVC_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=-1)),
            ])
MLP_One_vs_Rest = []
for category in categories:
    print('**Processing {} Tag...**'.format(category))
    
    # Training logistic regression model on train data
    LinearSVC_pipeline.fit(X_train, y_train[category])
    # calculating test accuracy
    prediction = LinearSVC_pipeline.predict(X_test)
    accuracy.append(accuracy_score(y_test[category], prediction))
    f1.append(f1_score(y_test[category], prediction))
    precision.append(precision_score(y_test[category], prediction))
    recall.append(recall_score(y_test[category], prediction))

**Processing java Tag...**
**Processing c# Tag...**
**Processing javascript Tag...**
**Processing python Tag...**
**Processing jquery Tag...**
**Processing html Tag...**
**Processing php Tag...**
**Processing android Tag...**
**Processing .net Tag...**
**Processing c++ Tag...**
**Processing sql Tag...**
**Processing css Tag...**
**Processing mysql Tag...**
**Processing string Tag...**
**Processing sql-server Tag...**
**Processing c Tag...**
**Processing git Tag...**
**Processing arrays Tag...**
**Processing asp.net Tag...**
**Processing linux Tag...**
**Processing regex Tag...**
**Processing iphone Tag...**
**Processing bash Tag...**
**Processing objective-c Tag...**
**Processing ios Tag...**
**Processing windows Tag...**
**Processing ruby Tag...**
**Processing eclipse Tag...**
**Processing tsql Tag...**
**Processing database Tag...**


In [ ]:
LinearSVC['f1'] = f1
LinearSVC['accuracy'] = accuracy
LinearSVC['precision'] = precision
LinearSVC['recall'] = recall

### MultiOutputClassifier

# Scores display

In [38]:
MLKNN_Score 

f1  accuracy  precision    recall
MLKNN  NaN   0.22214    0.45851  0.334151

In [39]:
def average_scores(series):
    average_scores = pd.DataFrame()
    average_f1_score = []
    average_accuracy = []
    average_precision = []
    average_recall = []
    average_f1_score.append(sum(series.f1)/len(series.f1))
    average_accuracy.append(sum(series.accuracy)/len(series.accuracy))
    average_precision.append(sum(series.precision)/len(series.precision))
    average_recall.append(sum(series.recall)/len(series.recall))
    average_scores['f1_score'] = average_f1_score
    average_scores['accuracy'] = average_accuracy
    average_scores['precision'] = average_precision
    average_scores['recall'] = average_recall
    return average_scores

In [40]:
average_log_reg = average_scores(Logistic_Regression)
average_svc = average_scores(SVC)
average_MNB = average_scores(Naive_Bayes)
average_MLP = average_scores(MLP)

In [41]:
average_log_reg

f1_score  accuracy  precision    recall
0  0.449659  0.967565   0.719561  0.354251

In [42]:
average_svc

f1_score  accuracy  precision    recall
0  0.434268   0.96846   0.745449  0.352039

In [43]:
average_MNB

f1_score  accuracy  precision    recall
0  0.240389  0.961487   0.741924  0.169793

In [44]:
average_MLP

f1_score  accuracy  precision    recall
0  0.000345    0.9549   0.033333  0.000174

In [45]:
Binary_relevance_score

Logistic_Regression       SVM  Naive_Bayes       MLP
f1                    0.475893  0.464338     0.263908  0.263908
accuracy              0.295083  0.309804     0.151995  0.151995
precision             0.747386  0.771194     0.798453  0.798453
recall                0.371581  0.365969     0.181323  0.181323

In [46]:
Label_powerset_score

Logistic_Regression       SVM  Naive_Bayes       MLP
f1                    0.525896  0.519203     0.434288  0.434288
accuracy              0.432357  0.430958     0.388415  0.388415
precision             0.611735  0.617244     0.594565  0.594565
recall                0.496949  0.488395     0.429116  0.429116